In [1]:
from etl.load_dataset import DatasetProcessor, get_tf_eggim_patch_dataset
from custom_models.cnns import base_resnet50
import tensorflow as tf
from keras.metrics import Precision, Recall, AUC, CategoricalAccuracy
from optimization.custom_losses import weighted_categorical_crossentropy
from custom_models.augmentation import basic_plus_color_augmentation, basic_augmentation
from custom_models.optimization_utilities import get_standard_callbacks
import numpy as np

2025-01-20 18:00:33.066367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737396033.081776   13502 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737396033.086369   13502 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 18:00:33.103329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
target_dir = '../2025-01_EGGIM_Dataset3'  # aqui foi o o que eu chamei no remote server, mas fica a criterio
patient_ids = np.load('configs/new_patient_ids.npy', allow_pickle=True)
dp = DatasetProcessor(target_dir)
df = dp.process()
togas_ids_boolean = np.array([x.startswith('PT') for x in df['patient_id'].values])
df_togas = df[togas_ids_boolean].reset_index(drop=True)
df_ipo = df[~togas_ids_boolean].reset_index(drop=True)


In [3]:
print(len(df.index),len(df_togas.index),len(df_ipo.index))

1280 866 414


In [4]:
df.shape

(1280, 6)

In [5]:
batch_size = 32
num_epochs = 100
learning_rate = 1e-4
num_folds = 5
n_classes =3

name = f'../logs/cv_patient_resnet_multi_{num_folds}'

In [6]:
split = dp.patient_wise_split_5_fold(df_togas,df_ipo,patient_ids,internal_train_size=0.9,target_variable='eggim_square',random_state=42)
i = 0
for fold, (df_train, df_val, df_test) in enumerate(split):
    tf_train_df = get_tf_eggim_patch_dataset(df_train,num_classes=3,
                                             augmentation_fn=basic_augmentation,
                                             preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    tf_val_df = get_tf_eggim_patch_dataset(df_val,
                                           num_classes=3,
                                           preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    tf_test_df = get_tf_eggim_patch_dataset(df_test,
                                            num_classes=3,
                                            preprocess_fn=tf.keras.applications.resnet.preprocess_input)
    y_train = df_train['eggim_square']
    class_counts = np.bincount(y_train)
    class_weights_manual = {i: len(y_train) / (len(class_counts) * class_counts[i]) for i in
                                range(len(class_counts))}
    weights = tf.constant(list(class_weights_manual.values()), dtype=tf.float32)

    tf_train_df = tf_train_df.batch(batch_size)
    tf_val_df = tf_val_df.batch(batch_size)
    tf_test_df = tf_test_df.batch(batch_size)

    n_classes = 3  # Replace with the number of classes you have
    model = base_resnet50(input_shape=(224, 224, 3), n_classes=n_classes)
        # Compile the model with Adam optimizer 13:21
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss=weighted_categorical_crossentropy(weights),
                      metrics=[CategoricalAccuracy(name='cat_accuracy'), Precision(name='precision'),
                               Recall(name='recall'),
                               AUC(name='auc')])

    name_fold = name + f'fold_{fold}'
    checkpoint_dir, callbacks = get_standard_callbacks(name_fold, learning_rate)
    model.fit(tf_train_df,
                  validation_data=tf_val_df,
                  epochs=num_epochs,
                  callbacks=callbacks)
    model.load_weights(f'{checkpoint_dir}/weights.weights.h5')
    model.evaluate(tf_test_df)
    
    

['PT053' 'PT016' 'PT049' 'PT074' 'PT045' 'PT081' 'PT004' 'PT030' 'PT082'
 'PT050' 'PT072' 'PT068' 'PT028']


I0000 00:00:1737396036.835950   13502 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9177 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/100


I0000 00:00:1737396044.598078   13577 service.cc:148] XLA service 0x7c987c002780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737396044.598092   13577 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-01-20 18:00:44.794180: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737396045.835389   13577 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-20 18:00:46.935697: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5046_0', 112 bytes spill stores, 224 bytes spill loads

2025-01-20 18:00:47.809123: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5053', 208 

 3/47 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - auc: 0.5236 - cat_accuracy: 0.3455 - loss: 1.2565 - precision: 0.3140 - recall: 0.2205

...


Epoch 1: val_loss improved from inf to 3.69907, saving model to ../logs/cv_patient_resnet_multi_5fold_3_20250120-181644/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 25s 393ms/step - auc: 0.6984 - cat_accuracy: 0.5249 - loss: 1.1658 - precision: 0.5668 - recall: 0.4449 - val_auc: 0.4192 - val_cat_accuracy: 0.3099 - val_loss: 3.6991 - val_precision: 0.3099 - val_recall: 0.3099 - learning_rate: 1.0000e-04
Epoch 2/100
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - auc: 0.5719 - cat_accuracy: 0.4144 - loss: 2.0120 - precision: 0.4160 - recall: 0.3975
Epoch 2: val_loss improved from 3.69907 to 2.19152, saving model to ../logs/cv_patient_resnet_multi_5fold_3_20250120-181644/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - auc: 0.5786 - cat_accuracy: 0.4203 - loss: 1.9824 - precision: 0.4221 - recall: 0.4031 - val_auc: 0.5152 - val_cat_accuracy: 0.3662 - val_loss: 2.1915 - val_precision: 0.3594 - val_recall: 0.3239 - learning_rate: 1.0000e-04
Epoch 3/100
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-01-20 18:21:58.000870: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1767_0', 8 bytes spill stores, 8 bytes spill loads

2025-01-20 18:21:58.200526: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1767', 168 bytes spill stores, 168 bytes spill loads

2025-01-20 18:21:58.475727: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1774', 312 bytes spill stores, 312 bytes spill loads

2025-01-20 18:21:58.662563: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1767', 16 bytes spill stores, 16 bytes spill loads

2025-01-20 18:21:58.721240: I external/local_xla/xla/str

5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - auc: 0.9277 - cat_accuracy: 0.7868 - loss: 0.5579 - precision: 0.8114 - recall: 0.7394
['PT006' 'PT065' 'PT080' 'PT024' 'PT025' 'PT038' 'PT022' 'PT062' 'PT079'
 'PT020' 'PT036' 'PT046' 'PT059']
Epoch 1/100
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - auc: 0.6540 - cat_accuracy: 0.4883 - loss: 1.2185 - precision: 0.5208 - recall: 0.3628

2025-01-20 18:22:14.493344: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5313', 24 bytes spill stores, 24 bytes spill loads

2025-01-20 18:22:14.645814: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5313', 24 bytes spill stores, 28 bytes spill loads

2025-01-20 18:22:14.667709: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5313', 32 bytes spill stores, 32 bytes spill loads



46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - auc: 0.6548 - cat_accuracy: 0.4893 - loss: 1.2222 - precision: 0.5217 - recall: 0.3649
Epoch 1: val_loss improved from inf to 4.16648, saving model to ../logs/cv_patient_resnet_multi_5fold_4_20250120-182202/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 18s 243ms/step - auc: 0.6556 - cat_accuracy: 0.4904 - loss: 1.2258 - precision: 0.5225 - recall: 0.3669 - val_auc: 0.3253 - val_cat_accuracy: 0.2083 - val_loss: 4.1665 - val_precision: 0.2083 - val_recall: 0.2083 - learning_rate: 1.0000e-04
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - auc: 0.5570 - cat_accuracy: 0.4342 - loss: 2.1078 - precision: 0.4489 - recall: 0.4196
Epoch 2: val_loss improved from 4.16648 to 2.75486, saving model to ../logs/cv_patient_resnet_multi_5fold_4_20250120-182202/weights.weights.h5
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - auc: 0.5600 - cat_accuracy: 0.4366 - loss: 2.0934 - precision: 0.4514 - recall: 0.4218 - val_auc: 0.3650 - val_cat_accuracy: 0.2222 - val